# Recommender Systems

installation problem but fixed with conda install -c conda-forge scikit-surprise

In [109]:
def cos_similarity(A,B):
    dot=np.dot(A,B)
    magnitude_A=np.linalg.norm(A)
    magnitude_B=np.linalg.norm(B)
    return (dot/(magnitude_A*magnitude_B))

for the embeddings i have (256 columns and 158901 rows) as we have 158901 entities in a 256 embedding space

In [110]:
import numpy as np 
import csv
import pandas as pd
#import embeddings
embeddings=np.load('data/entity_embeds.npy')
embeddings_df=pd.DataFrame(embeddings)
max=40000
print(embeddings_df[:max])
embeddings_dict={}
iter=0
for i in range(0,max):
    embeddings_dict[i]=embeddings_df[0][i]

print("embeddings_dict",embeddings_dict.items())
entity_dict = {}
with open('data/entity_ids.del', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter='\t')
    for row in reader:
        if len(row) == 2:
            key, value = row[0], row[1]
            entity_dict[key] = value
print(entity_dict['0'])
print(list(entity_dict.items())[:10])

              0           1           2            3           4    \
0       13.605913 -120.160194 -157.706177   207.169342 -160.651230   
1     -188.857727 -346.554016  106.492256  1232.438110 -362.230713   
2     -413.393280 -314.701324  119.153091  1205.572266 -510.052856   
3     -146.255875 -446.803680   51.218224   506.433441 -248.142456   
4     -167.102737 -325.045898  -41.878307  1220.312744 -379.709808   
...           ...         ...         ...          ...         ...   
39995   41.783798 -149.400375  -61.449875   177.626404 -382.049500   
39996 -127.613037 -363.610352 -123.911804   235.235886 -309.252228   
39997 -153.222031 -119.612617   57.536541  1202.419800   -8.032053   
39998 -280.673553 -348.666840  -42.977905  1340.181030 -244.858871   
39999 -284.179016 -236.959976  -21.441145  1238.909180 -361.720856   

              5           6           7           8           9    ...  \
0     -443.186554  328.840820 -167.950256  -99.612396 -288.122864  ...   
1     -229.

idea: we have the uri and thereby the index with which we have to compute the 

In [70]:
similarity_score={}
iter=0
compare_id=34515
embeddings_df=np.transpose(embeddings_df)
for element in embeddings_df:
    similarity_score[iter]=cos_similarity(embeddings_df[compare_id],embeddings_df[iter])
    iter+=1
similarity_dict=sorted(similarity_score.items(),key=lambda item: item[1], reverse=True)
similarity_dict=similarity_dict[1:]


In [87]:
top_similar_dict={}
for i in range(0,20): 
    top_similar_dict[list(entity_dict.items())[similarity_dict[i][0]][0]]=list(entity_dict.items())[similarity_dict[i][0]][1],similarity_dict[i][1]

print(top_similar_dict['19517'][0])

http://www.wikidata.org/entity/Q184768


In [98]:
for id in top_similar_dict.keys():
    qid = top_similar_dict[id][0]
    if qid in rating_dict:
        # Add rating as a new entry
        top_similar_dict[id] = top_similar_dict[id] + (rating_dict[qid],)
print(top_similar_dict)

{'19517': ('http://www.wikidata.org/entity/Q184768', np.float32(0.9101844)), '5298': ('http://www.wikidata.org/entity/Q1196157', np.float32(0.8601065)), '13280': ('http://www.wikidata.org/entity/Q3225260', np.float32(0.8473474)), '27047': ('http://www.wikidata.org/entity/Q202326', np.float32(0.84606326)), '41972': ('http://www.wikidata.org/entity/Q657079', np.float32(0.80261976)), '30877': ('http://www.wikidata.org/entity/Q220735', np.float32(0.7911662)), '10505': ('http://www.wikidata.org/entity/Q182692', np.float32(0.7890977), 10.0, 10.0), '16468': ('http://www.wikidata.org/entity/Q458656', np.float32(0.78868073)), '6146': ('http://www.wikidata.org/entity/Q570567', np.float32(0.7879312)), '1294': ('http://www.wikidata.org/entity/Q607179', np.float32(0.7848752)), '11669': ('http://www.wikidata.org/entity/Q201674', np.float32(0.78405434)), '6089': ('http://www.wikidata.org/entity/Q3558150', np.float32(0.78268284)), '116896': ('http://www.wikidata.org/entity/Q2995837', np.float32(0.7825

now we create a rating, similarity score that would recommend the best rated similar movies. 80% similarity and 20% rating

In [ ]:
for id in top_similar_dict.keys():
    base_tuple = top_similar_dict[id]
    if len(base_tuple) > 3:
        continue  
    
    similarity_score = base_tuple[1]  
    if len(base_tuple) > 2 and base_tuple[2] is not None:
        rating = base_tuple[2]
        weighted_score = 0.8 * similarity_score + 0.2 * (rating / 10)
        top_similar_dict[id] = base_tuple + (weighted_score,)
    else:
        top_similar_dict[id] = base_tuple + (similarity_score,)

sorted_dict = dict(
    sorted(
        top_similar_dict.items(),
        key=lambda x: x[1][2] if x[1][2] is not None else -float('inf'),
        reverse=True
    )
)

for k, v in sorted_dict.items():
    print(f"{k}: {v}")

10505: ('http://www.wikidata.org/entity/Q182692', np.float32(0.7890977), 10.0, 10.0, np.float32(0.8312782), np.float32(0.8312782))
19517: ('http://www.wikidata.org/entity/Q184768', np.float32(0.9101844), np.float32(0.9101844), np.float32(0.7463512))
5298: ('http://www.wikidata.org/entity/Q1196157', np.float32(0.8601065), np.float32(0.8601065), np.float32(0.7052874))
13280: ('http://www.wikidata.org/entity/Q3225260', np.float32(0.8473474), np.float32(0.8473474), np.float32(0.6948249))
27047: ('http://www.wikidata.org/entity/Q202326', np.float32(0.84606326), np.float32(0.84606326), np.float32(0.6937719))
41972: ('http://www.wikidata.org/entity/Q657079', np.float32(0.80261976), np.float32(0.80261976), np.float32(0.65814817))
30877: ('http://www.wikidata.org/entity/Q220735', np.float32(0.7911662), np.float32(0.7911662), np.float32(0.64875627))
16468: ('http://www.wikidata.org/entity/Q458656', np.float32(0.78868073), np.float32(0.78868073), np.float32(0.6467182))
6146: ('http://www.wikidata

import ratings, since there were problems loading the file in using csv reader it has to be done manually

In [96]:
import re
import pandas as pd
qid_list = []
rating_list = []
sentiment_list = []
comment_list = []

with open('data/user-comments.csv', 'r', encoding='utf-8') as file:
    header = file.readline().strip()
    for line in file:
        match = re.split(r',(?=(?:[^"]*"[^"]*")*[^"]*$)', line.strip())
        if len(match) == 4:
            qid_list.append(match[0])
            rating_list.append(match[1])
            sentiment_list.append(match[2])
            comment_list.append(match[3])

ratings = pd.DataFrame({
    'qid': qid_list,
    'rating': rating_list,
    'sentiment': sentiment_list,
    'comment': comment_list
})

ratings['rating'] = pd.to_numeric(ratings['rating'], errors='coerce')
ratings['sentiment'] = pd.to_numeric(ratings['sentiment'], errors='coerce')
print(ratings.head())

ratings['entity_id'] = pd.factorize(ratings['qid'])[0]
movie_mapping = ratings[['entity_id', 'qid']]
movie_mapping = movie_mapping.drop_duplicates(subset='qid', keep='first')
movie_mapping = movie_mapping.sort_values(by='entity_id')
print(movie_mapping)

mapping_dict = movie_mapping.set_index('qid')['entity_id'].to_dict()
rating_dict = ratings.set_index('qid')['rating'].to_dict()


                                       qid  rating  sentiment  \
0   http://www.wikidata.org/entity/Q780951    10.0        1.0   
1  http://www.wikidata.org/entity/Q1171313     4.0        0.0   
2  http://www.wikidata.org/entity/Q1195571     8.0        1.0   
3   http://www.wikidata.org/entity/Q467695     1.0        0.0   
4  http://www.wikidata.org/entity/Q1755064     7.0        1.0   

                                             comment  
0  ..especially by Lambert. This is the essential...  
1  I am still waiting for that dark muppet film w...  
2  DaniReese Witherspoon has always been very clo...  
3  This could be a cute movie for kids My grandso...  
4  Burt Reynolds stars as an undercover cop who i...  
      entity_id                                      qid
0             0   http://www.wikidata.org/entity/Q780951
1             1  http://www.wikidata.org/entity/Q1171313
2             2  http://www.wikidata.org/entity/Q1195571
3             3   http://www.wikidata.org/entity/Q4

weighted average of rating and similarity